<a href="https://colab.research.google.com/github/martin-dj/casa0018/blob/main/anomaly_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

First set up the necessary Python imports and set up Tensor Board which provides a visual output of the training process.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime, os

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.preprocessing import MinMaxScaler
# TimeseriesGenerator is now located in tensorflow.keras.utils instead of keras.preprocessing.sequence
from tensorflow.keras.utils import timeseries_dataset_from_array # Updated import
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN,LSTM
from keras.callbacks import EarlyStopping

# Load the TensorBoard notebook extension
#%load_ext tensorboard
%reload_ext tensorboard


# Clear any logs from previous runs
%rm -rf ./logs/

# Create Data

Create sine function data. We’ll use the NumPy linspace to generate x values ranging between 0 and 60 and NumPy sine function to generate sine values to the corresponding x. We also add a linear componenent to the sine data to generate a trend.

We then add some noise to the data. Finally, we visualize the data.

In [ ]:
x = np.linspace(0, 60, 601)  # in radians
y = np.sin(x) + 0.02*x

# Plot our data
plt.figure(figsize=(12,8))
plt.plot(x,y)
plt.show()

y += 0.2*np.random.randn(*y.shape) #Add some noise to the data to make it more realistic

# Plot our data
plt.figure(figsize=(12,8))
plt.plot(x, y, 'b.')
plt.show()

# Define a dataframe using x and y values.
df = pd.DataFrame(data=y,index=x,columns=['Sine'])


# Split Data
Split the data into train and validate subsets. The train dataset is used for training the model and the validate data set is used to validate the model against unseen data as it undergoes training. Normally there is a third unseen data set used for testing the trained model. However, the time series used here is sufficiently predictable to test 'by eye'.

First, we’ll check the length of the data frame and use a fraction of the data to validate our model. Now if we multiply the length of the data frame with val_percent and round the value (as we are using for indexing purpose) we’ll get the index position i.e., val_index. Last, we’ll split the train and validation data using the val_index.

In [ ]:
val_pecent = 0.16667
len(df)*val_pecent
val_point = np.round(len(df)*val_pecent)
val_index = int(len(df) - val_point)
train = df.iloc[:val_index]
val = df.iloc[val_index:]

plt.figure(figsize=(12,8))
plt.xlim(0, 60)

print(len(df)) # 601
print(len(train))
print(len(val))

plt.plot(train)
plt.plot(val)


# Normalise
We need to normalise the data in the range 0-1. We use a scaler to determine the max and min of the complete data set and then use the scaler to scale both the training a validation data set

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df)
MinMaxScaler(copy=True, feature_range=(0, 1))
scaled_train = scaler.transform(train)
scaled_val = scaler.transform(val)

# Time Series Generator
One problem we’ll face when using time series data is, we must transform the data into sequences of samples with input data and target data before feeding it into the model. We should select the length of the data sequence (window length) in such a way so that the model has an adequate amount of input data to generalize and predict i.e. in this situation we must feed the model at least with one cycle of sine wave values.

The model takes the previous 50 data points (one window) as input data and uses it to predict the next point, which is then compared to the actual target value for backpropagation and gradient descent.

This process is time-consuming and difficult if we perform this manually, hence we’ll make use of the Keras Timeseries Generator which transforms the data automatically and ready to train models without heavy lifting.

We can see that the length of the scaled_train is 501 and the length of the generator is 451(501–50) i.e. if we perform the tuple unpacking of the generator function using X,y as variables, X comprises the 50 data points (training window) and y contains the 51st data point which the model uses for the prediction target. i.e. X0 = values at timesteps 0-49 and y0 = value at timestep 50; X1 = values at timesteps 1-50 and y1 = value at timestep 51; ... X451 = values at timesteps 451-500 and y451 = value at timestep 501.

We also create a validation_generator that operates on the validation data set (scaled_val in the code). The validator_generator is not used to train the model. Instead, it is used after each epoch to validate how well the current model fits an **unseen** data set.

The batch size indicates how many sequences (windows) are seen by the network before the network weights are updated. A batch size of 1 means the weights are updated after every sequence (window) is input to the network. In this case a batch size of 10 is used, which speeds up training.

An epoch is one complete pass through the training data set.

In [ ]:
length = 30 # sequence length - the length of the training window
batch_size = 5

# Using timeseries_dataset_from_array to create a generator
generator = timeseries_dataset_from_array(data=scaled_train, targets=scaled_train, sequence_length=length, batch_size=batch_size)

validation_generator = timeseries_dataset_from_array(scaled_val, scaled_val, sequence_length=length, batch_size=batch_size)

print(len(scaled_train)) # 501
# the generator object has no length attribute
# it is possible to determine the length, but this is more involved
# the below code will demonstrate how to find the length if required
print(len(list(generator.as_numpy_iterator())))

# Create a model

The code to create a LSTM is similar to that for earlier NNs you have already seen.

The variable (n_features) defined stands for the number of features in the training data i.e., as we are dealing with univariate data we’ll only have one feature whereas if we are using multivariate data containing multiple features then we must specify the count of features in our data.

In [ ]:
n_features = 1

# Encoder
encoder = Sequential()
encoder.add(LSTM(length, return_sequences=True, input_shape=(length, n_features)))

# Decoder
decoder = Sequential()
decoder.add(LSTM(length, return_sequences=False, input_shape=(length, length))) # Pass the output shape of the encoder
decoder.add(Dense(1))

# Autoencoder
autoencoder = Sequential([encoder, decoder])
autoencoder.compile(loss='mse', optimizer='adam')

In [ ]:
autoencoder.summary()
autoencoder.fit(generator, epochs=100, validation_data=validation_generator)

# Comparing Forecasts Using The Validation Data

Now let's compare the forecasts of the LSTM model using our validation data set.

Again remembering this is not a true test as the validation set was used to validate our models.

Before we plot the results we reverse the scaling transformations.

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-length:]

current_batch = first_eval_batch.reshape(1, length, n_features)


for i in range(len(val)):
  current_pred = autoencoder.predict(current_batch)[0]

  test_predictions.append(current_pred)

  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis = 1)

true_predictions = scaler.inverse_transform(test_predictions)
val['LSTM Predictions'] = true_predictions
val.plot(figsize=(12,8))